### Preliminaries

The main challenge in computing this quantity will be retaining numerical precision, since our motivating hypothesis is that $p(s_L \textrm{ recalled correctly}|M, N, q)$ might often be very close to unity. In fact, it might even be exponentially close to unity, so it could be useful to think instead in terms of the error, and specifically its logarithm. We can adjust our calculation accordingly by noting:

$$p(s_L \textrm{ recalled incorrectly}|M, N, q) = 1 - p(s_L \textrm{ recalled correctly}|M, N, q) \\ \approx \cfrac{1}{N_{MC}}\sum\limits_{V_1, ..., V_{2L}}(1 - p(s_L \textrm{ recalled correctly}|V_1, ..., V_{2L}, M, N, q)) \leq \\
\cfrac{1}{N_{MC}}\sum\limits_{V_1, ..., V_{2L}}\left[1 - f(V_1, ..., V_{2L})h(V_1, ..., V_{2L}, N, q)^{M-2L}\right].$$

Taking the logarithm we then have:

$$\log p(s_L \textrm{ recalled incorrectly}|M, N, q) \leq \\
-\log N_{MC} + \log\sum\limits_{V_1, ..., V_{2L}}\left[1 - f(V_1, ..., V_{2L})h(V_1, ..., V_{2L}, N, q)^{M-2L}\right].$$

However, if any of terms in the sum is rounded to zero from numerical imprecision, the entire sum will be evaluated to be $-\infty$, preventing us from exploring the dependence of error on $N, q, M$ and $L$ when it is already very low. To deal with this, we will instead calculate the logarithm of the terms inside the sum, along with the fact that even though there's not an obviously useful mathematical identity expressing the logarithm of a sum as a function of a the logarithm of its individual terms, computationally this can be done very practically while still avoiding significant numerical errors (see appendix).

To determine

$$\log\left[1 - f(V_1, ..., V_{2L})h(V_1, ..., V_{2L}, N, q)^{M-2L}\right]$$

we first recall that $f$ is always either $0$ or $1$, so when it is $0$ this quantity will equal $0$. When $f = 1$ (which hypothesize to often be true) we must consider two numerical cases: (1) $h^{M-2L}$ is sufficiently less than unity that numerical errors are insignificant, or (2) $h^{M-2L}$ is close enough to unity that numerical errors might cause a problem.

In case (1) we can simply calculate $\log\left[1 - h^{M-2L}\right]$. In case (2) we must remember that we are fundamentally trying to get an accurate description of how close $h^{M - 2L}$ is to $1$, and since it is presumably very close, we would like to use the logarithm to maintain accuracy.

To proceed, we use the following Taylor expansion, valid when $1 - x < \epsilon$, i.e., $x$ is very close to unity:

$$1 - x \approx -\log x \implies \log(1 - x) \approx \log(-\log x) $$

or in our specific situation, since in case (2) we have that $h^{M-2L}$ is very close to unity,

$$\log(1 - h^{M-2L}) \approx \log(-\log h^{M-2L}) = \log\left(-\log\left(\prod\limits_{i=1}^{2L}c_i\right)^{M-2L}\right) = \\
\log\left(-(M - 2L)\left(\sum\limits_{i=1}^{2L}\log c_i\right)\right) =
\log(M-2L) + \log\left(-\left(\sum\limits_{i=1}^{2L}\log c_i\right)\right).$$

The final issue we must address is that $c_i$ may be so close to unity that numerical errors in calculating its log will cause us to lose desired precision. When this is the case, i.e., when $c_i > 1 - \epsilon$, it can be more practical to use the log of the *survival function* $\log s_i$, where $s_i = 1 - c_i$ but can be calculated without any significant loss of precision when $c_i > 1 - \epsilon$. To use this to our advantage, we note that when $c_i > 1 - \epsilon$ we have

$$\log c_i = \log(1 - s_i) \approx -s_i.$$

Thus, when calculating the inner sum we simply need to replace $\log c_i$ by $-s_i$ whenever $c_i > 1 - \epsilon$. This final adjustment allows us to calculate $\log(1 - h^{M-2L})$ without any significant loss of accuracy. And when all is said and done the calculation of our upper bound on the log error

$$\log p(s_L \textrm{ recalled incorrectly}|M, N, q)$$

can be completed in $O(N_{MC}L)$.

Remark: we can get a rough estimate for the log error $\log(1 - h^{M-2L})$ by noticing that the survival function, which is a stand-in for $-\log c_i$, will decrease roughly exponentially with $N$, such that the error equation is roughly:

$$\log(M - 2L) + \log(\alpha^N) = \log(M - 2L) - \alpha'N$$

since $\alpha < 1$.
This means that to keep the error under a fixed amount, M should be able to grow exponentially faster than $N$.

### Pseudocode

We can summarize the final algorithm for calculating the upper bound on $\log p(s_L \textrm{ recalled incorrectly}|M, N, q)$ as follows.

#### Principal routines

```python
LOG_EPSILON = -9*log(10)

def max_items_low_error(max_log_error, N_MC, N, L, q, R=None):
    """
    Calculate the (log) max number of items that a set of
    associative memory units can support.
    """
    
    fs, log_neg_log_hs = calc_fs_and_log_neg_log_hs_asym(N_MC, N, L, q, R)
    
    # define function to be solved
    def func_to_solve(log_m):
        """Increasing function of log_m."""
    
        log_m_minus_2l = log_m if log(2*l) - log_m < LOG_EPSILON else log(exp(log_m) - 2*l)

        log_sum_terms = np.nan * np.zeros(len(vs))
        approx_mask = log_m_minus_2l + log_neg_log_h < LOG_EPSILON

        log_sum_terms[approx_mask] = log_m_minus_2l + log_neg_log_hs[approx_mask]
        h_to_m_minus_2l = exp(-exp(log_neg_log_h[~approx_mask] + log_m_minus_2l))
        log_sum_terms[~approx_mask] = log(1 - h_to_m_minus_2l)

        f_0_mask = (fs == 0)
        log_sum_terms[f_0_mask] = 0
        
        return log_sum(log_sum_terms) - max_log_error
        
    # determine initial optimization bracket
    log_m_ub = log(2L)
    if func_to_solve(log_m_ub) >= 0: return -inf
    
    log_m_ub += 1
    while func_to_solve(log_m_ub) < 0: log_m_ub += 1
    
    log_m_lb = log_m_ub - 1
    
    # find the root of the equation
    
    return brentq(func_to_solve, log_m_lb, log_m_ub)
    

def log_upper_error_bound(log_ms, N_MC, N, L, q, R=None):
    """
    Calculate the log of the upper bound on the expected error
    for several different numbers of item units.
    """

    fs, log_neg_log_hs = calc_fs_and_log_neg_log_hs_asym(N_MC, N, L, q, R)
        
    # calculate the log of each term in the sum,
    # using approximations where necessary
    
    errors = []
    
    for log_m in log_ms:
    
        log_m_minus_2l = log_m if log(2*l) - log_m < LOG_EPSILON else log(exp(log_m) - 2*l)

        log_sum_terms = np.nan * np.zeros(len(vs))

        approx_mask = log_m_minus_2l + log_neg_log_h < LOG_EPSILON

        log_sum_terms[approx_mask] = log_m_minus_2l + log_neg_log_hs[approx_mask]
        h_to_m_minus_2l = exp(-exp(log_neg_log_h[~approx_mask] + log_m_minus_2l))
        log_sum_terms[~approx_mask] = log(1 - h_to_m_minus_2l)

        f_0_mask = (fs == 0)
        log_sum_terms[f_0_mask] = 0
    
        errors.append(log_sum(log_sum_terms))
        
    return errors
```

#### Subroutines

```python
def calc_fs_and_log_neg_log_hs(N, L, q, N_MC, R):

    if R == None: # symmetric case
    
        vs = sample_vs(N_MC, N, L, q)
        xs_all, rs_all = zip(*[calc_xs_and_rs(v) for v in vs])
        
    else: # asymmetric case
    
        vs, us = sample_vs_and_us(N_MC, N, L, q, R)
        xs_all, rs_all = zip(*[calc_xs_and_rs(v, u) for v, u in zip(vs, us)])
    
    x_sizes_all = [np.sum(xs, axis=1) for xs in xs_all]
    
    fs = [calc_f(v, xs, rs) for v, xs, rs in zip(vs, xs_all, rs_all)]
    log_neg_log_hs = [
        calc_log_neg_log_h(x_sizes, rs, q) 
        for x_size, rs in zip(x_sizes_all, rs_all)
    ]
    
    return np.array(fs), np.array(log_neg_log_hs)
    
def sample_vs(n_mc, n, l, q):

    return (np.random.rand(n_mc, 2*l, n) < q).astype(int)
    
def sample_vs_and_us(n_mc, n, l, q, R):

    vs = sample_vs(n_mc, n, l, q)
    d = (1 - q*R) / (1 - q)
    us[vs.astype(bool)] = (np.random.rand(vs.sum()) < q*R).astype(int)
    us[~vs.astype(bool)] = (np.random.rand(vs.sum()) < q*d).astype(int)
    
    return vs, us
    
def calc_xs_and_rs(v, u=None):

    # calculate pairwise intersections
    if u is None: isctns = [v[ctr] * v[ctr + 1] for ctr in range(0, len(v), 2)]
    else: isctns = [u[ctr] * u[ctr + 1] for ctr in range(0, len(u), 2)]
        
    isctns = np.array([val for pair in zip(isctns, isctns) for val in pair])
    
    # calculate maintained set and xs
    maintained = np.sum(isctns, axis=0)
    
    xs = [np.sum(v[ctr] * maintained) for ctr in range(len(v))]
    rs = isctns.sum(axis=1)
    
    return xs, rs
    
def calc_f(v, xs, rs):

    if len(v) <= 2: return 1
    
    for ctr_0, (x, r) in enumerate(zip(xs, rs)):
        
        # pair to which item belongs
        pair = (ctr_0 - 1, ctr_0) if x % 2 else (ctr_0, ctr_0 + 1)
        
        for ctr_1 in [j for j in range(len(vs)) if j not in pair]:
        
            if (v[ctr_1] * x).sum() >= r: return 0
            
    return 1
    
def calc_log_neg_log_h(x_sizes, rs, q):

    # calculate the log survival function for each x, r
    log_sfs = binom_log_sf(rs, x_sizes, q)
    
    # replace log_sfs by log(-log_cdf) when log_sf is too big for taylor approx
    mask = log_sfs > LOG_EPSILON
    log_sfs[mask] = log(-binom_log_cdf(rs[mask], x_sizes[mask], q))
    
    # calculate log of sum of sfs in terms of log_sfs
    return log_sum(log_sfs)

def log_sum(log_xs):

    ... calculate the logarithm of a sum given the logarithms of its terms 
```

### Code tests

How to convince oneself the code works:

Potentially testable functions:

* `calc_log_mc_sum`
    * provide some example values and make sure approximation works correctly
* `calc_fs_and_log_neg_log_hs`
    * not a good way to test this, but it's fairly simple so we can trust it...
* `sample_vs`
    * test with small and large `q`
* `sample_vs_and_us`
    * test with small and large `q` and with R = 0, 1/q, 1/2q
* `calc_f`
    * test with a few examples
* `calc_log_neg_log_hs`
    * test with a few examples in the case of using the approximation and not
* `calc_xs_and_rs`
    * test with examples when u = None and otherwise